### Hourly forecasting of energy meter readings on BDG2 dataset

- historical data = 1 week (168 data points)
- forecast horizon = 1 day (24 data points)

**Loading TimesFM Model**

In [1]:
import os
import glob
import time
from datetime import datetime
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import islice

from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg

import warnings
warnings.filterwarnings('ignore') 

In [2]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      #examples["country"].append(country)
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      #examples["gen_forecast"].append(sub_df["gen_forecast"][start:context_end + horizon_len].tolist())
      #examples["week_day"].append(sub_df["week_day"][start:context_end + horizon_len].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end + horizon_len)])

    return examples

In [ ]:
# Benchmark
batch_size = 32
context_len = 168
horizon_len = 24

def process_building(df):
    building_name = df.columns[0]
    df.columns = ['y']
    input_data = get_batched_data_fn(df, batch_size=500)
    # print(input_data)
    
    windows_all = []
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(input_data['inputs_ts'], 
                                                      input_data['inputs'], 
                                                      input_data['outputs_ts'], 
                                                      input_data['outputs']):
        
        input_df = pd.DataFrame({'timestamp': inputs_ts, 
                                 'target': inputs})
        
        output_df = pd.DataFrame({'timestamp': outputs_ts, 
                                 'target': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['item_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        windows_all.append(combined)
        
    windows_all_df = pd.concat(windows_all)
    windows_all_df.timestamp = pd.to_datetime(windows_all_df.timestamp)
    windows_all_df.set_index('timestamp', inplace=True)

    return windows_all_df

In [ ]:
def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])
    df.index = pd.to_datetime(df.index)
    df['month'] = df.index.month
    training_set = df[df.month <= 6]
    test_set = df[df.month > 6]
    training_set = training_set.drop(columns='month')
    test_set = test_set.drop(columns='month')
    df = df.drop(columns='month')
    

    print(f'fine-tune set date range: {training_set.index[0]} {training_set.index[-1]}, '
      f'test set date range: {test_set.index[0]} {test_set.index[-1]}')
            

    if df.shape[1] < 2:
        return None
        
    print(datetime.now(), df.shape, flush=True)

    results_all = []
    c =0
    lag = 168 
    for building_name in df.columns:
        print(f'{datetime.now()} {c} / {len(df.columns)} {building_name}', flush=True)

        windowed_df_train = process_building(training_set[[building_name]])
        windowed_df_test = process_building(test_set[[building_name]])

        forecaster = ForecasterAutoreg(
                    regressor        = LGBMRegressor(max_depth=-1, n_estimators=100, n_jobs=24, verbose=-1),
                    lags             = 168
                )
        forecaster.fit(y= windowed_df_train['target'],
            )

        p = []
        for i in windowed_df_test.item_id_no.unique():#(pred_days):
            # i -= 1           
            seq_ptr =lag + 24 * i
        
            df_test = windowed_df_test[windowed_df_test.item_id_no == i]
            last_window  = df_test.iloc[0:168]
            ground_truth = df_test.iloc[168:192]
        
            predictions = forecaster.predict(
                steps       = 24,
                last_window = last_window['target']
            )
            # p.append(predictions)
            res = ground_truth.copy()
            res = res[['target']]
            # print(res)
            res.columns = ['y_true']
            res = res.reset_index()
            res.insert(2, 'y_pred', predictions.reset_index()['pred'])
            res.set_index('timestamp', inplace=True)
            # res['y_pred'] = predictions
            p.append(res)
        res = pd.concat(p)
        res['building'] = building_name
        results_all.append(res)
        c+=1
        # if i == 2:
        #    break
        #break
        
    results_all_df = pd.concat(results_all)
    return results_all_df

In [5]:
files_list = glob.glob('/home/user/New_Buildings_Datasets/Mathura_and_Bareilly/dataverse_files/processed/Bareilly/*csv')

dataset = 'Bareilly'
os.makedirs(f'forecasts/{dataset}/', exist_ok = True)
os.makedirs(f'results/{dataset}/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)
    if results is not None:
        results.to_csv(f'forecasts/{dataset}/{os.path.basename(filename)}')
    print('')

2024-11-13 10:56:40.478578 /home/user/New_Buildings_Datasets/Mathura_and_Bareilly/dataverse_files/processed/Bareilly/Bareilly_2021.csv
fine-tune set date range: 2021-01-01 00:00:00 2021-06-30 23:00:00, test set date range: 2021-07-01 00:00:00 2021-10-31 23:00:00
2024-11-13 10:56:40.513205 (7296, 38)
2024-11-13 10:56:40.513757 0 / 38 BR02
2024-11-13 10:56:43.145358 1 / 38 BR04
2024-11-13 10:56:45.707183 2 / 38 BR05
2024-11-13 10:56:48.139152 3 / 38 BR06
2024-11-13 10:56:50.688980 4 / 38 BR08
2024-11-13 10:56:53.322238 5 / 38 BR09
2024-11-13 10:56:56.075422 6 / 38 BR11
2024-11-13 10:56:58.579607 7 / 38 BR12
2024-11-13 10:57:01.257759 8 / 38 BR13
2024-11-13 10:57:03.749265 9 / 38 BR15
2024-11-13 10:57:06.249225 10 / 38 BR16
2024-11-13 10:57:08.687360 11 / 38 BR18
2024-11-13 10:57:11.421166 12 / 38 BR19
2024-11-13 10:57:13.860240 13 / 38 BR22
2024-11-13 10:57:16.395851 14 / 38 BR24
2024-11-13 10:57:18.931725 15 / 38 BR27
2024-11-13 10:57:21.375114 16 / 38 BR28
2024-11-13 10:57:23.885590 17

### Metrics

In [34]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import root_mean_squared_log_error
from permetrics.regression import RegressionMetric

dataset = 'Bareilly'
files_list = glob.glob(f'forecasts/{dataset}/*.csv')

metrics_all_files = []

for filename in files_list:
    print(filename)
    res = pd.read_csv(filename)
    metrics_all = []
    for (g, data) in res.groupby(['building']):
        data = data.dropna()
        data = data[data.y_pred >= 0]
        print(g[0]) 
        # print(data)
        if not data.empty:
            rmse= root_mean_squared_error(data.y_true, data.y_pred)
            mae= mean_absolute_error(data.y_true, data.y_pred)
            mape = mean_absolute_percentage_error(data.y_true, data.y_pred)
            mse= mean_squared_error(data.y_true, data.y_pred)
            msle= mean_squared_log_error(data.y_true, data.y_pred)
            rmsle= root_mean_squared_log_error(data.y_true, data.y_pred)
            nrmse = rmse / (data.y_true.mean()) 
    
            evaluator = RegressionMetric(data.y_true.to_list(), data.y_pred.to_list())
            nrmse_eve = evaluator.normalized_root_mean_square_error()
            evaluator = RegressionMetric(data.y_true.to_list(), data.y_pred.to_list())
            smape= evaluator.symmetric_mean_absolute_percentage_error()
        
            metrics = pd.DataFrame({'building_name': [g[0]], 
                               'mae': [mae],
                                'mape': [mape],
                               'mse': [mse], 'rmse': [rmse], 'msle': [msle], 'rmsle': [rmsle], 'nrmse' : [nrmse],
                                  'nrmse_eve':[nrmse_eve] , 'sMAPE' : [smape]})
            metrics_all.append(metrics)
        else:
            continue
    
    metrics_all_df = pd.concat(metrics_all)
    metrics_all_df.to_csv(f'results/{dataset}/{os.path.basename(filename)}')

    metrics_all_df['filename'] = os.path.basename(filename)
    metrics_all_files.append(metrics_all_df)

metrics_all_files_df = pd.concat(metrics_all_files)

forecasts/Bareilly/Bareilly_2021.csv
BR02
BR04
BR05
BR06
BR08
BR09
BR11
BR12
BR13
BR15
BR16
BR18
BR19
BR22
BR24
BR27
BR28
BR29
BR30
BR31
BR32
BR33
BR34
BR35
BR36
BR37
BR38
BR39
BR42
BR43
BR44
BR45
BR46
BR48
BR49
BR50
BR51
BR52
forecasts/Bareilly/Bareilly_2020.csv
BR02
BR03
BR04
BR05
BR06
BR07
BR08
BR09
BR10
BR11
BR12
BR13
BR14
BR15
BR16
BR17
BR18
BR19
BR20
BR22
BR23
BR24
BR26
BR27
BR28
BR29
BR30
BR31
BR32
BR33
BR34
BR35
BR36
BR37
BR38
BR39
BR42
BR43
BR44
BR45
BR46
BR48
BR49
BR50
BR51
BR52


In [35]:
metrics_all_files_df.to_csv(f'results/{dataset}/results_combined.csv')
metrics_all_files_df

,building_name,mae,mape,mse,rmse,msle,rmsle,nrmse,nrmse_eve,sMAPE,filename
0,BR02,0.117476,5.290643e+14,0.022705,0.150683,0.018512,0.136059,inf,1.596809,1.000000,Bareilly_2021.csv
0,BR04,0.253858,2.017607e+14,0.121779,0.348968,0.055623,0.235846,0.878858,1.187571,0.493030,Bareilly_2021.csv
0,BR05,0.056564,2.427426e+14,0.004381,0.066192,0.003883,0.062314,14.735510,2.035608,0.985217,Bareilly_2021.csv
0,BR06,0.374824,3.849457e+13,0.301620,0.549200,0.075101,0.274047,0.718456,1.255064,0.228024,Bareilly_2021.csv
0,BR09,0.023424,1.054911e+14,0.000573,0.023935,0.000559,0.023647,inf,4.865613,1.000000,Bareilly_2021.csv
...,...,...,...,...,...,...,...,...,...,...,...
0,BR48,0.035598,4.368681e+13,0.005245,0.072421,0.004021,0.063410,1.486228,0.887391,0.634778,Bareilly_2020.csv
0,BR49,0.305062,1.035871e+14,0.220498,0.469572,0.063898,0.252781,0.819685,0.936455,0.257329,Bareilly_2020.csv
0,BR50,0.117040,1.147711e+14,0.031723,0.178109,0.018782,0.137047,0.961200,1.045028,0.375844,Bareilly_2020.csv
0,BR51,0.468614,2.089392e+14,0.474969,0.689180,0.110342,0.332177,0.723400,0.954650,0.259446,Bareilly_2020.csv


In [36]:
metrics_all_files_df.describe()*100

,mae,mape,mse,rmse,msle,rmsle,nrmse,nrmse_eve,sMAPE
count,8100.000000,8.100000e+03,8100.000000,8100.000000,8100.000000,8100.000000,8000.000000,8.100000e+03,8100.000000
mean,17.830885,1.314066e+16,12.204773,26.721432,3.807939,16.392725,inf,7.394877e+15,47.464099
std,15.891634,1.169417e+16,17.894096,22.644490,4.374047,10.652386,NaN,6.655390e+16,26.793928
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,39.035324,6.787996e+01,14.453952
25%,6.142700,5.164622e+15,1.013156,10.065565,0.744342,8.627528,72.876882,9.930356e+01,27.300730
50%,11.747587,1.070402e+16,3.172288,17.810917,1.735000,13.171940,89.099668,1.187571e+02,35.855788
75%,28.444223,1.710728e+16,15.508130,39.380363,6.365549,25.230039,139.957687,1.362373e+02,62.566802
max,60.737949,6.883830e+16,64.970245,80.604122,17.081493,41.329762,inf,5.989851e+17,100.000000
